In [ ]:
! pip install contractions

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import nltk

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

import contractions
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
disneyland_df = pd.read_csv('../content/drive/MyDrive/project_3/Data/disneyland.csv')
universalstudios_df = pd.read_csv('../content/drive/MyDrive/project_3/Data/universalstudios.csv')

In [ ]:
combined_df= pd.concat([disneyland_df,universalstudios_df])

In [ ]:
combined_df.head()

,Unnamed: 0,name,subreddit,title,selftext,upvote_ratio,num_comments,created_utc
0,0,t3_1brgdb7,disneyparks,My friends just went to MK for the first time ...,Quote: “I was disappointed in Gaston’s Tavern....,0.10,3,1.711801e+09
1,1,t3_1br7ntk,disneyparks,"D23 Parks Bingo sheet, do you think the option...",NaN,0.83,7,1.711768e+09
2,2,t3_1bqyshu,disneyparks,Olu Mel! 💚,NaN,0.68,0,1.711745e+09
3,3,t3_1bqd0kg,disneyparks,CookieAnn!! 🤎,NaN,0.76,5,1.711676e+09
4,4,t3_1bqcac7,disneyparks,Incredible parade video,The YouTube algorithm gave me this astonishing...,0.67,2,1.711674e+09


In [ ]:
combined_df.isnull().sum()

Unnamed: 0        0
name              0
subreddit         0
title             0
selftext        500
upvote_ratio      0
num_comments      0
created_utc       0
dtype: int64

### Data Cleaning

In [ ]:
#https://www.analyticsvidhya.com/blog/2022/01/text-cleaning-methods-in-nlp/

# Drop Unnamed:0
combined_df.drop(columns='Unnamed: 0',inplace=True)

# change 'disneyparks' to 1 and 'universalstudios' to 0
combined_df['subreddit'] = combined_df['subreddit'].map({'disneyparks': 1, 'universalstudios':0})


# change created_utc to data time format
combined_df['created_utc'] = combined_df['created_utc'].apply(lambda x:datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))


# Create Year and Month column

combined_df['year'] = combined_df['created_utc'].str.split('-').str[0]
combined_df['month'] = combined_df['created_utc'].str.split('-').str[1]

# Fill selftext na value with blank
combined_df['selftext'] = combined_df['selftext'].fillna(' ')


# create columns: len of title, len of selftext, world count of title, world count of selftext
combined_df['title_len'] = combined_df['title'].apply(lambda  x: len(x))
combined_df['selftext_len'] = combined_df['selftext'].apply(lambda  x: len(x))

combined_df['title_word_count'] = combined_df['title'].apply(lambda  x: len(x.split()))
combined_df['selftext_word_count'] = combined_df['selftext'].apply(lambda  x: len(x.split()))

combined_df['title_selftext'] = combined_df['title'] + ' ' + combined_df['selftext']

In [ ]:
combined_df.head(3)

,name,subreddit,title,selftext,upvote_ratio,num_comments,created_utc,year,month,title_len,selftext_len,title_word_count,selftext_word_count,title_selftext
0,t3_1brgdb7,1,My friends just went to MK for the first time ...,Quote: “I was disappointed in Gaston’s Tavern....,0.10,3,2024-03-30,2024,03,93,455,19,84,My friends just went to MK for the first time ...
1,t3_1br7ntk,1,"D23 Parks Bingo sheet, do you think the option...",,0.83,7,2024-03-30,2024,03,67,1,12,0,"D23 Parks Bingo sheet, do you think the option..."
2,t3_1bqyshu,1,Olu Mel! 💚,,0.68,0,2024-03-29,2024,03,10,1,3,0,Olu Mel! 💚


### Text Cleaning

In [ ]:
# Normalize to lower case
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda  x: x.lower())

# Replace 'mk with magic kingdom'
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda x: re.sub('mk', 'magic kingdom', x))

# Replace 'hk with hong kong '
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda x: re.sub('hk', 'hong kong', x))

# Replace 'dlp with disneyland park'
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda x: re.sub('dlp', 'disneyland park', x))

# Removing Contractions
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda x: contractions.fix(str(x)))


# Removing Website Links from title and selftext

# Define regex pattern to match website URLs
pattern = r"https?://(?:www\.)?\S+"

combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda  x: re.sub(pattern,'',x))


# Removing Punctuations, number, emoji, accept only text
pattern = r'[^a-zA-Z\s]'

combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda  x: re.sub(pattern,'',x))

# Removing Extra Space
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda  x: x.split())

# Remove Stop Word
stop_words = set(stopwords.words('english'))

# Append 'would' to the set of stopwords - based on EDA in 02_EDA
stop_words.update(['would','go','went','gone'])

def remove_stop_words(string):
    filtered_words = [word for word in string if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_words


combined_df['title_selftext'] = combined_df['title_selftext'].apply(remove_stop_words)




In [ ]:
# Lemmatizing Data
lemmatizer = WordNetLemmatizer()

def lemmatize_list(w_list):
    lem_word = []
    for word in w_list:
        words = nltk.word_tokenize(word)
        lem_word.append(lemmatizer.lemmatize(word))
    return lem_word

combined_df['title_selftext'] = combined_df['title_selftext'].apply(lemmatize_list)

In [ ]:
# convert back to line
combined_df['title_selftext'] = combined_df['title_selftext'].apply(lambda x: ' '.join(x))

# for training in fasttext model later on
combined_df['subreddit_title_selftext'] = "__label__" + combined_df['subreddit'].astype(str) + ' ' + combined_df['title_selftext']

combined_df['title_selftext_len'] = combined_df['selftext'].apply(lambda  x: len(x))
combined_df['title_selftext_word_count'] = combined_df['title_selftext'].apply(lambda  x: len(x.split()))

In [ ]:
combined_df.head(1)

,name,subreddit,title,selftext,upvote_ratio,num_comments,created_utc,year,month,title_len,selftext_len,title_word_count,selftext_word_count,title_selftext,subreddit_title_selftext,title_selftext_len,title_selftext_word_count
0,t3_1brgdb7,1,My friends just went to MK for the first time ...,Quote: “I was disappointed in Gaston’s Tavern....,0.1,3,2024-03-30,2024,03,93,455,19,84,friend magic kingdom first time disappointed r...,__label__1 friend magic kingdom first time dis...,455,43


In [ ]:
#Save to csv file - for EDA
combined_df.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_df.csv', index=False)

### Create a test and test dataset

In [ ]:
# Test
test_df = combined_df.sample(frac=0.20, random_state=42)

# Train
train = combined_df.drop(test_df.index)

# Further split train dataset to train and valid
train_df, valid_df = train_test_split(train,test_size=0.2, random_state=42)

test_df.shape,train_df.shape,valid_df.shape

((398, 17), (1040, 17), (260, 17))

### For Fast Text format

In [ ]:
train_df.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_train_df',columns=['subreddit_title_selftext'],index=False,header=False)
test_df.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_test_df',columns=['subreddit_title_selftext'],index=False,header=False)
valid_df.to_csv('../content/drive/MyDrive/project_3/Data/cleaned_combined_valid_df',columns=['subreddit_title_selftext'],index=False,header=False)